In [15]:
import gtsam
from gtsam import Point2, Pose2, symbol
import numpy as np
import math
import time 
import plotly.express as px

from gtsam.utils import plot # Needs matplotlib
from numpy.random import default_rng

rng = default_rng()

NM = gtsam.noiseModel

In [3]:
odometry_data = {}
data_file_dr = gtsam.findExampleDataFile("Plaza2_DR.txt")
odo_times_list = []
for row in np.loadtxt(data_file_dr):
    t, distance_traveled, delta_heading = row
    odometry_data[t] = Pose2(distance_traveled, 0, delta_heading)
    odo_times_list.append(t)

odo_times_list.sort() # Ensure times are sorted
M = len(odometry_data)
print(f"Read {M} odometry entries.")

Read 4090 odometry entries.


In [4]:
triples = []
landmark_ids = set()
data_file_td = gtsam.findExampleDataFile("Plaza2_TD.txt")
for row in np.loadtxt(data_file_td):
    t, sender, receiver, _range = row
    landmark_id = int(receiver)
    triples.append((t, landmark_id, _range))
    landmark_ids.add(landmark_id)

K = len(triples)
sorted_landmark_ids = sorted(list(landmark_ids))
print(f"Read {K} range triples for {len(landmark_ids)} unique landmarks.")

Read 1816 range triples for 4 unique landmarks.


In [5]:
sigmaR = 50        # range standard deviation (matches MATLAB)
sigmaInitialLandmark = 1.0 # Stddev for random landmark init (matches MATLAB)
robust = True

In [6]:
# Set Noise parameters (mirrors MATLAB script)
priorSigmas = gtsam.Point3(1, 1, math.pi)  # Prior on Pose 0
pointPriorSigmas = gtsam.Point2(1, 1) # Loose prior on landmarks
odoSigmas = gtsam.Point3(0.05, 0.01, 0.2) # Odometry noise

priorNoise = NM.Diagonal.Sigmas(priorSigmas) # prior on pose 0
pointPriorNoise = NM.Diagonal.Sigmas(pointPriorSigmas) # loose LM prior
odoNoise = NM.Diagonal.Sigmas(odoSigmas)     # odometry

# Range noise
gaussian = NM.Isotropic.Sigma(1, sigmaR)     # non-robust
base = NM.mEstimator.Tukey(15) # Tukey M-estimator like MATLAB
tukey = NM.Robust.Create(base, gaussian)  # robust
rangeNoise = tukey if robust else gaussian

In [7]:
# Initialize Factor Graph and Initial Values
graph = gtsam.NonlinearFactorGraph()
initial = gtsam.Values()

# Add prior on first pose
# Estimate pose0 from the first odometry time's neighborhood or use a fixed reasonable guess.
# Using the same value as the iSAM example for consistency.
pose0 = Pose2(-34.2086489999201, 45.3007639991120, math.pi - 2.021089)
graph.add(gtsam.PriorFactorPose2(0, pose0, priorNoise))
initial.insert(0, pose0)

# Initialize Landmarks (similar to MATLAB approach)
print("Initializing Landmarks...")
for j in sorted_landmark_ids:
    landmark_key = symbol('L', j)
    # Initialize randomly around origin (adjust scale if needed)
    initial_point = Point2(sigmaInitialLandmark * rng.normal(), sigmaInitialLandmark * rng.normal())
    initial.insert(landmark_key, initial_point)
    # Optional: Add a loose prior to help localization if landmarks are poorly constrained
    # graph.add(gtsam.PriorFactorPoint2(landmark_key, Point2(0,0), pointPriorNoise))
print(f"Initialized {len(sorted_landmark_ids)} landmarks.")

Initializing Landmarks...
Initialized 4 landmarks.


In [8]:
print("\nBuilding Pose Chain and Odometry Factors...")
lastPose = pose0
pose_times = [odo_times_list[0]] # Approximate time for pose 0
# Loop over odometry measurements
for i, t in enumerate(odo_times_list, 1):
    # get odometry measurement for this time step
    odometry = odometry_data[t]

    # add odometry factor
    graph.add(gtsam.BetweenFactorPose2(i - 1, i, odometry, odoNoise))

    # predict pose and add as initial estimate
    predictedPose = lastPose.compose(odometry)
    lastPose = predictedPose
    initial.insert(i, predictedPose)
    pose_times.append(t)

print(f"Added {M} odometry factors and initial pose estimates.")


Building Pose Chain and Odometry Factors...
Added 4090 odometry factors and initial pose estimates.


In [9]:
print("\nAdding Range Factors...")
# Convert pose_times to numpy array for efficient search
pose_times_np = np.array(pose_times)

# Add range factors
range_factors_added = 0
for t_range, landmark_id, measured_range in triples:
    # Find the pose index active at the time of the range measurement
    # Get the index of the *last* pose time that is <= t_range
    pose_index = np.searchsorted(pose_times_np, t_range, side='right') - 1
    
    # Ensure pose_index is valid (it might be -1 if t_range is before the first odometry time)
    if pose_index < 0:
        # print(f"Warning: Range measurement at time {t_range} is before the first pose time {pose_times_np[0]}. Skipping.")
        pose_index = 0 # Or skip the measurement

    landmark_key = symbol('L', landmark_id)
    graph.add(gtsam.RangeFactor2D(pose_index, landmark_key, measured_range, rangeNoise))
    range_factors_added += 1

print(f"Added {range_factors_added} range factors.")


Adding Range Factors...
Added 1816 range factors.


In [10]:
# Graph built, optimize!
print("\nOptimizing the factor graph...")
start_time = time.time()

params = gtsam.LevenbergMarquardtParams()
params.setRelativeErrorTol(1e-3) # Convergence tolerance
params.setVerbosityLM("SUMMARY") # Print LM progress
optimizer = gtsam.LevenbergMarquardtOptimizer(graph, initial, params)
result = optimizer.optimize()

end_time = time.time()
print(f"Optimization complete in {end_time - start_time:.2f} seconds.")

print(f"Initial Error: {graph.error(initial)}")
print(f"Final Error: {graph.error(result)}")


Optimizing the factor graph...
Optimization complete in 8.03 seconds.
Initial Error: 298.291739023152
Final Error: 0.8770075415268919
Initial error: 298.292, values: 4095
iter      cost      cost_change    lambda  success iter_time
   0       139725     -1.4e+05      1e-05      1       0.16
iter      cost      cost_change    lambda  success iter_time
   0      8.9e+03     -8.6e+03     0.0001      1        0.2
iter      cost      cost_change    lambda  success iter_time
   0      1.9e+02      1.1e+02      0.001      1       0.12
   1      2.7e+03     -2.5e+03     0.0001      1       0.23
   1      1.1e+02           82      0.001      1       0.18
   2      1.7e+03     -1.6e+03     0.0001      1       0.26
   2           84           22      0.001      1       0.26
   3      5.9e+02     -5.1e+02     0.0001      1       0.08
   3           75          8.7      0.001      1       0.09
   4      2.8e+02       -2e+02     0.0001      1       0.08
   4           66            9      0.001    

In [11]:
# Print optimized landmarks:
print("\nOptimized Landmark Locations:")
for j in [0, 1, 5, 6]: # Print specific landmarks like MATLAB example
    if j in sorted_landmark_ids:
        landmark_key = gtsam.symbol('L', j)
        try:
            p = result.atPoint2(landmark_key)
            print(f"  L{j}: {p}")
        except Exception as e:
            print(f"  L{j}: Not found in result ({e})")
    else:
        print(f"  L{j}: Not present in TD data.")


Optimized Landmark Locations:
  L0: [-48.11371377  31.9206595 ]
  L1: [-80.93080228  53.94011755]
  L5: [-47.58464216 -19.97679349]
  L6: [-17.70794095  66.35831441]


In [12]:
# Extract poses and landmarks for plotting
poses_result = gtsam.utilities.allPose2s(result)
landmarks_result = gtsam.utilities.extractPoint2(result)
positions_result = np.array([poses_result.atPose2(key).translation() 
                             for key in range(M + 1)]) # Poses 0 to M

# Also extract initial estimate poses (dead reckoning)
poses_initial = gtsam.utilities.allPose2s(initial)
positions_initial = np.array([poses_initial.atPose2(key).translation() 
                              for key in range(M + 1)])

print(f"\nExtracted {positions_result.shape[0]} final poses and {landmarks_result.shape[0]} landmarks.")


Extracted 4091 final poses and 4 landmarks.


In [33]:
# Plot using Plotly (similar to iSAM example)
fig = px.line(x=positions_initial[:,0], y=positions_initial[:,1], title='Plaza2 Batch SLAM Result')
fig.data[0].name = 'Initial (Odometry)'
fig.data[0].showlegend = True
fig.data[0].line.color = 'orange'
fig.data[0].line.dash = 'dash'

fig.add_scatter(x=positions_result[:,0], y=positions_result[:,1], mode='lines', 
                line=dict(color='black'), name='Optimized Path')

fig.add_scatter(x=landmarks_result[:,0], y=landmarks_result[:,1], mode='markers', 
                marker=dict(color='red', symbol='star', size=8), name='Landmarks')

# Configure layout
fig.update_layout(margin=dict(l=20, r=20, t=40, b=20), 
                  legend=dict(yanchor="top", y=0.99, xanchor="left", x=0.01),
                  xaxis_title="X (m)", yaxis_title="Y (m)")
fig.update_yaxes(scaleanchor="x", scaleratio=1) # Ensure aspect ratio is 1:1
import plotly.io as pio
pio.renderers.default = "browser"
fig.show()

In [47]:
# Write the initial (odometry) robot poses to data/range/unoptimized.txt for simulation replay (shifted so first x,y is 0,0)
import os
os.makedirs('data', exist_ok=True)
x0, y0 = positions_initial[0]
with open('data/range/unoptimized.txt', 'w') as f:
    for i in range(positions_initial.shape[0]):
        x, y = positions_initial[i]
        x = (x - x0) / 10
        y = (y - y0) / 10
        # Try to get heading if available (from poses_initial)
        try:
            theta = poses_initial.atPose2(i).theta()
        except Exception:
            theta = 0.0
        f.write(f"{x} {y} {theta}\n")
print("Unoptimized poses written to data/range/unoptimized.txt (shifted and scaled by 1/10)")

Unoptimized poses written to data/range/unoptimized.txt (shifted and scaled by 1/10)


In [48]:
# Write the optimized robot poses to data/range/optimized.txt for simulation replay (shifted so first x,y is 0,0)
import os
os.makedirs('data', exist_ok=True)
x0_opt, y0_opt = positions_result[0]
with open('data/range/optimized.txt', 'w') as f:
    for i in range(positions_result.shape[0]):
        x, y = positions_result[i]
        x = (x - x0_opt) / 10
        y = (y - y0_opt) / 10
        # Try to get heading if available (from poses_result)
        try:
            theta = poses_result.atPose2(i).theta()
        except Exception:
            theta = 0.0
        f.write(f"{x} {y} {theta}\n")
print("Optimized poses written to data/range/optimized.txt (shifted and scaled by 1/10)")

Optimized poses written to data/range/optimized.txt (shifted and scaled by 1/10)


In [49]:
# Write optimized landmark positions to file
os.makedirs('data', exist_ok=True)
with open('data/range/landmarks_optimized.txt', 'w') as f:
    for lx, ly in landmarks_result:
        rlx = (lx - x0_opt) / 10
        rly = (ly - y0_opt) / 10
        f.write(f"{rlx} {rly}\n")
print("Optimized landmark positions written to data/range/landmarks_optimized.txt")

Optimized landmark positions written to data/range/landmarks_optimized.txt


In [51]:
landmarks_initial = gtsam.utilities.extractPoint2(initial)
# Write unoptimized landmark positions to file
os.makedirs('data', exist_ok=True)
with open('data/range/landmarks_unoptimized.txt', 'w') as f:
    for lx, ly in landmarks_initial:
        rlx = (lx - x0_opt) / 10
        rly = (ly - y0_opt) / 10
        f.write(f"{rlx} {rly}\n")
print("Unoptimized landmark positions written to data/range/landmarks_unoptimized.txt")

Unoptimized landmark positions written to data/range/landmarks_unoptimized.txt
